<a href="https://colab.research.google.com/github/PadmarajBhat/Real-Time-Analytics-on-Hadoop/blob/master/Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -U pyspark

Requirement already up-to-date: pyspark in /usr/local/lib/python3.6/dist-packages (2.4.3)


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MLlib").getOrCreate()
a = [(0,1),(0,2),(1,1),(1,2),(2,1),(2,2),(3,1),(3,2)]
b = spark.createDataFrame(a,["class","num"])
b.show()

+-----+---+
|class|num|
+-----+---+
|    0|  1|
|    0|  2|
|    1|  1|
|    1|  2|
|    2|  1|
|    2|  2|
|    3|  1|
|    3|  2|
+-----+---+



In [0]:
b= b.withColumn("num", b['num']/b['num'])
b.show()

+-----+---+
|class|num|
+-----+---+
|    0|1.0|
|    0|1.0|
|    1|1.0|
|    1|1.0|
|    2|1.0|
|    2|1.0|
|    3|1.0|
|    3|1.0|
+-----+---+



In [0]:
b=b.withColumn("new", b['class']+b['num'])
b.show()

+-----+---+---+
|class|num|new|
+-----+---+---+
|    0|1.0|1.0|
|    0|1.0|1.0|
|    1|1.0|2.0|
|    1|1.0|2.0|
|    2|1.0|3.0|
|    2|1.0|3.0|
|    3|1.0|4.0|
|    3|1.0|4.0|
+-----+---+---+



In [5]:

b.show()

+-----+---+---+
|class|num|new|
+-----+---+---+
|    0|1.0|1.0|
|    0|1.0|1.0|
|    1|1.0|2.0|
|    1|1.0|2.0|
|    2|1.0|3.0|
|    2|1.0|3.0|
|    3|1.0|4.0|
|    3|1.0|4.0|
+-----+---+---+



In [7]:
!echo $PATH

/usr/local/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin


In [8]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2300.000
BogoMIPS:            4600.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd i

In [10]:
!less /proc/cpuinfo

processor       : 0
vendor_id       : GenuineIntel
cpu family      : 6
model           : 63
model name      : Intel(R) Xeon(R) CPU @ 2.30GHz
stepping        : 0
microcode       : 0x1
cpu MHz         : 2300.000
cache size      : 46080 KB
physical id     : 0
siblings        : 2
core id         : 0
cpu cores       : 1
apicid          : 0
initial apicid  : 0
fpu             : yes
fpu_exception   : yes
cpuid level     : 13
wp              : yes
:^C
